# HW3
## Sajad Rahmanian
## 97101683

## Imports

In [1]:
import numpy as np

## Connected Components
For finding CCs of and undirected graph we use [DFS](https://en.wikipedia.org/wiki/Depth-first_search "Wikipedia") algorithm. And the test result shows that it works correctly.

In [2]:
class UG():
    def __init__(self, GG):
        self.V = GG.shape[0]
        self.G = GG.copy()
        self.adj = [[] for i in range(GG.shape[0])]
        self.visitFlag = [False for i in range(GG.shape[0])]
        self.Components = []

    def DFS(self, Component, v):
        # Setting V as a visited node and adding it to the Component:
        self.visitFlag[v] = True
        Component.append(v)
        # Finding adjacents of v and running DFS for them in order to complete the Component:
        for adjacent in self.adj[v]:
            if self.visitFlag[adjacent] == False:
                self.DFS(Component, adjacent)
        
        return Component
    
    def FindAdjacents(self):
        # Filling the adj list which holds adjacents of each node:
        for u in range(self.V):
            for w in range(self.V):
                if self.G[u, w] == 1:
                    self.adj[u].append(w)
                    
    def ConnectedComponents(self):
        self.FindAdjacents()
        
        for v in range(self.V):
            if self.visitFlag[v] == True:
                continue
            
            component = []
            cc = self.DFS(component, v)
            self.Components.append(cc)
        
        return self.Components
            

In [3]:
# Testing UG:
G = np.array([[0,1,0,0,0,0,0,0],[1,0,0,0,0,0,0,0],[0,0,0,1,0,0,0,0], [0,0,1,0,0,0,0,0], [0,0,0,0,0,1,0,0],[0,0,0,0,1,0,1,0],[0,0,0,0,0,1,0,1], [0,0,0,0,0,0,1,0]])
cc = UG(G).ConnectedComponents()
cc

[[0, 1], [2, 3], [4, 5, 6, 7]]

## Implementing *ChainCom* algorithm

In [4]:
def ChainCom(U, v):
    A = {v}
    B = {i for i in range(U.shape[0]) if i != v}
    G = U.copy()
    O = []
    while len(B) != 0:
        T = set()
        # Finding nodes of B that are adjacent to any node in A and inserting them into T: 
        for w in B:
            for a in A:
                if (G[w, a] == 1) & (G[a, w] == 1):
                    T.update([w])
        # Orienting all edges between A and T:
        for c in A:
            for t in T:
                if (G[c, t] == 1) & (G[t, c] == 1):
                    G[t, c] = 0
        
        # Orienting y-z for y,z,x of the form x->y-z
        keepRunning = True
        while keepRunning:
            keepRunning = False
            for t in T:
                for j in T:
                    if (G[t, j] == 1) & (G[j, t] == 1):
                        for x in range(U.shape[0]):
                            if (G[x, t] == 1) & (G[t, x] == 0) & (G[x, j] == 0) & (G[j, x] == 0):
                                G[j, t] = 0
                                keepRunning = True
        # Updating A, B
        A = T
        B = B.difference(T)
        # Inserting all chain components of G_T into O
        G1 = G * G.T
        T = list(T)
        T.sort()
        G1 = G1[np.array(T)[:, None], np.array(T)]
        cc = UG(G1).ConnectedComponents()
        for c in cc:
            c.sort()
            c = np.array(c)
            O += [G1[c[:, None], c]]
            
    return G, O

## Implementing *SizeMEC* algorithm

In [5]:
def SizeMEC(U):
    # Determining number of nodes and edges of the input UCCG
    p = U.shape[0]
    n = U.sum() / 2
    # Base graphs sizemec
    if n == p - 1:
        return p
    elif n == p:
        return 2 * p
    elif n == p * (p - 1) / 2 - 2:
        return (p ** 2 - p - 4) * np.math.factorial(p - 3)
    elif n == p * (p - 1) / 2 - 1:
        return 2 * np.math.factorial(p - 1) - np.math.factorial(p - 2)
    elif n == p * (p - 1) / 2:
        return np.math.factorial(p)
    
    # Recursive part of the algorithm
    out = 0
    for j in range(p):
        s = 1
        G, O = ChainCom(U, j)
        for uu in O:
            s *= SizeMEC(uu)
        out += s
    return out

## Testing the algorithms
The algorithm has been tested for three UCCGs and answered correcly. The first graph is from the paper that intrduced these algorithms.
The second graph is from *Active Learning of Causal Networks
with Intervention Experiments and Optimal Designs* by Yang-Bo He and Zhi Geng and all of its 12 dags are presented there. And the third graph is an easy one since it could be originated from all combinations except for one that yields a v-structure. So its number of dags is 4 - 1 = 3.

In [6]:
A = np.array([[0, 1, 1, 0, 0], [1, 0, 1, 1, 1], [1, 1, 0, 0, 1], [0, 1, 0, 0, 1], [0, 1, 1, 1, 0]])
SizeMEC(A)

14

In [7]:
B = np.array([[0, 1, 1, 0, 0], [1, 0, 1, 1, 0], [1, 1, 0, 1, 0], [0, 1, 1, 0, 1], [0, 0, 0, 1, 0]])
SizeMEC(B)

12

In [8]:
C = np.array([[0, 1, 0], [1, 0, 1], [0, 1, 0]])
SizeMEC(C)

3